In [1]:
import random
import time
import warnings
import argparse
import shutil
import os.path as osp

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.optim import SGD
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data import DataLoader
import torch.nn.functional as F
import cv2
import numpy as np
import utils
from torch.utils.data import Dataset, DataLoader
from tllib.alignment.dan import MultipleKernelMaximumMeanDiscrepancy, ImageClassifier
from tllib.modules.kernels import GaussianKernel
from tllib.utils.data import ForeverDataIterator
from tllib.utils.metric import accuracy
from tllib.utils.meter import AverageMeter, ProgressMeter
from tllib.utils.logger import CompleteLogger
from tllib.utils.analysis import collect_feature, tsne, a_distance
import pandas as pd
import matplotlib.pyplot as plt


In [4]:
#import torch module
import torch
#create  tensor matrix
data1 = torch.tensor([[2.0,4.0,5.0,3.2], [3.4,5.6,7.8,9.0], [2.1,3.2,4.3,5.6], [5.4,3.2,2.3,7.8]])  

#display
print("Actual Tensor matrix: ")
print(data1)

print("Pseudo Inverse matrix: ")
#return Pseudo inverse of the above matrix
print(torch.pinverse(data1))

Actual Tensor matrix: 
tensor([[2.0000, 4.0000, 5.0000, 3.2000],
        [3.4000, 5.6000, 7.8000, 9.0000],
        [2.1000, 3.2000, 4.3000, 5.6000],
        [5.4000, 3.2000, 2.3000, 7.8000]])
Pseudo Inverse matrix: 
tensor([[-0.3627,  3.0709, -5.4110,  0.4902],
        [ 1.4628, -5.8971,  9.2172, -0.4133],
        [-0.7418,  4.0696, -6.3933,  0.1987],
        [-0.1303, -0.9067,  1.8498, -0.1002]])


In [5]:
#import torch module
import torch
 
 
#create  tensor matrix
data1 = torch.tensor([[2.0,4.0,5.0,3.2],[3.4,5.6,7.8,9.0], [2.1,3.2,4.3,5.6], [5.4,3.2,2.3,7.8]])  

#display
print("Actual Tensor matrix: ")
print(data1)

print("Inverse matrix: ")
#return inverse of the above matrix
print(torch.inverse(data1))

Actual Tensor matrix: 
tensor([[2.0000, 4.0000, 5.0000, 3.2000],
        [3.4000, 5.6000, 7.8000, 9.0000],
        [2.1000, 3.2000, 4.3000, 5.6000],
        [5.4000, 3.2000, 2.3000, 7.8000]])
Inverse matrix: 
tensor([[-0.3627,  3.0709, -5.4110,  0.4902],
        [ 1.4628, -5.8971,  9.2172, -0.4132],
        [-0.7418,  4.0696, -6.3933,  0.1987],
        [-0.1303, -0.9067,  1.8498, -0.1002]])


In [6]:
data1

tensor([[2.0000, 4.0000, 5.0000, 3.2000],
        [3.4000, 5.6000, 7.8000, 9.0000],
        [2.1000, 3.2000, 4.3000, 5.6000],
        [5.4000, 3.2000, 2.3000, 7.8000]])

In [2]:
df = pd.read_feather('/home/bkcs/HDD/FL/Data_Processing/Data/Concatenated/Concatenated_small/Train/Concatenated_train_256.feather')

KeyError: 0

In [ ]:
def split_data(df, frac=0.2):
    seletected = df['flow_id'].drop_duplicates().sample(frac=frac)

    val = df[df['flow_id'].isin(seletected)]
    train = df[~df['flow_id'].isin(seletected)]
    return train, val


In [ ]:
def source_target_split(df, frac=0.5):
    df_label_3 = df[df.Label == 2]
    seletected_label_3 = df_label_3['flow_id'].drop_duplicates().sample(
        frac=0.99)
    seletected = df['flow_id'].drop_duplicates().sample(frac=frac)

    source_select = seletected[~seletected.isin(seletected_label_3)]

    source = df[df['flow_id'].isin(source_select)]
    target = df[~df['flow_id'].isin(seletected)]
    return source, target


In [ ]:
def resize_image(image, target_size=(224, 224)):
    return cv2.resize(image, target_size, interpolation=cv2.INTER_LINEAR)


In [ ]:
def most_frequent(List):
    return max(set(List), key=List.count)


In [ ]:
class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]


In [ ]:
def data_processing(raw_data):
    # Get flow label
    result = raw_data.groupby('flow_id')['Label'].apply(list).to_dict()
    flow_label = []
    for flow in result:
        flow_label.append(most_frequent(result[flow]))
    flow_label = np.array(flow_label)
    # Reshape payloads
    true_data = raw_data.drop('flow_id', axis=1)
    datas = true_data.drop('Label', axis=1).to_numpy()/255
    datas = datas.reshape(-1, 20, 256).astype('float32')
    # Resize each image in the dataset
    datas = np.array([resize_image(img) for img in datas])
    rgb_datas = np.repeat(datas[:, :, np.newaxis,], 3, axis=2)
    print("before " + str(rgb_datas.shape))
    rgb_datas = np.moveaxis(rgb_datas, 2, 1)
    print("after " + str(rgb_datas.shape))
    final_dataset = MyDataset(rgb_datas, flow_label)
    return final_dataset


In [ ]:
Train_data = pd.read_feather(
    '/home/bkcs/HDD/FL/Data_Processing/Capture_small/Train/Capture_data_20_256.feather')
Test_data = pd.read_feather(
    '/home/bkcs/HDD/FL/Data_Processing/Capture_small/Test/Capture_test_20_256.feather')


In [ ]:
print(Train_data.shape[0]/20)
print(Test_data.shape[0]/20)


In [ ]:
Train_data.head(10)


In [ ]:
Test_data.head()


In [ ]:
label_mapping = {1: 0, 2: 0, 3: 0, 4: 1, 5: 1}

source_labels = [1, 3, 5]
target_labels = [2, 4]


In [ ]:

train_source = Train_data.loc[Train_data['Label'].isin(source_labels)]
train_target = Train_data.loc[Train_data['Label'].isin(target_labels)]


In [ ]:
train_source.head(10)


In [ ]:
def remapping(df,map):
    df_copy = df.copy()
    df_copy['Label'] = df_copy['Label'].replace(map)
    return df_copy

In [ ]:
train_source = remapping(train_source,label_mapping)


In [ ]:
train_target_copy = train_target.copy()
train_target_copy['Label'] = train_target_copy['Label'].replace(label_mapping)
train_target = train_target_copy
del train_target_copy

In [ ]:
train_source.head(10)


In [ ]:

val_raw = Test_data.loc[Test_data['Label'].isin(source_labels)]
test_raw = Test_data.loc[Test_data['Label'].isin(target_labels)]


In [ ]:
val_raw.head()

In [ ]:
val_raw['Label'] = val_raw.Label.replace(label_mapping)
test_raw['Label'] = test_raw.Label.replace(label_mapping)
